**Connect google drive**

In [1]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Wed Dec  8 22:21:53 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8    26W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

**1) Clone the Darknet**



In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15376, done.
remote: Total 15376 (delta 0), reused 0 (delta 0), pack-reused 15376
Receiving objects: 100% (15376/15376), 13.98 MiB | 16.87 MiB/s, done.
Resolving deltas: 100% (10341/10341), done.


**2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Configure Darknet network for training YOLO V3**

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=64/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"


mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [ ]:
!echo "Ear" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2021-12-08 16:31:32--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  57.1MB/s    in 2.7s    

2021-12-08 16:31:35 (57.1 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
!unzip /mydrive/yolov3/images.zip -d data/obj


Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/0001.png       
 extracting: data/obj/0001.txt       
  inflating: data/obj/0002.png       
 extracting: data/obj/0002.txt       
  inflating: data/obj/0003.png       
 extracting: data/obj/0003.txt       
  inflating: data/obj/0004.png       
 extracting: data/obj/0004.txt       
  inflating: data/obj/0005.png       
 extracting: data/obj/0005.txt       
  inflating: data/obj/0006.png       
 extracting: data/obj/0006.txt       
  inflating: data/obj/0007.png       
 extracting: data/obj/0007.txt       
  inflating: data/obj/0008.png       
 extracting: data/obj/0008.txt       
  inflating: data/obj/0009.png       
 extracting: data/obj/0009.txt       
  inflating: data/obj/0010.png       
 extracting: data/obj/0010.txt       
  inflating: data/obj/0011.png       
 extracting: data/obj/0011.txt       
  inflating: data/obj/0012.png       
  inflating: data/obj/0012.txt       
  inflating: data/obj/0013.png       
  inflating: 

In [ ]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, float(numbers[1])/480, float(numbers[2])/360, float(numbers[3])/480, float(numbers[4])/360)
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

0 177 68 20 35 

['0', '177', '68', '20', '35']
0 data/obj/0701.txt
0 0.36875 0.18888888888888888 0.041666666666666664 0.09722222222222222
0 336 65 14 36 

['0', '336', '65', '14', '36']
0 data/obj/0701.txt
0 0.7 0.18055555555555555 0.029166666666666667 0.1
0 367 144 60 81 

['0', '367', '144', '60', '81']
1 data/obj/0397.txt
0 0.7645833333333333 0.4 0.125 0.225
0 320 149 31 44 

['0', '320', '149', '31', '44']
2 data/obj/0320.txt
0 0.6666666666666666 0.41388888888888886 0.06458333333333334 0.12222222222222222
0 325 98 46 55 

['0', '325', '98', '46', '55']
3 data/obj/0462.txt
0 0.6770833333333334 0.2722222222222222 0.09583333333333334 0.1527777777777778
0 429 119 21 55 

['0', '429', '119', '21', '55']
4 data/obj/0373.txt
0 0.89375 0.33055555555555555 0.04375 0.1527777777777778
0 305 181 59 68 

['0', '305', '181', '59', '68']
5 data/obj/0299.txt
0 0.6354166666666666 0.5027777777777778 0.12291666666666666 0.18888888888888888
0 231 136 9 39 

['0', '231', '136', '9', '39']
6 data/obj/0

In [ ]:
import glob
images_list = glob.glob("data/obj/*.png")
print(images_list)

['data/obj/0279.png', 'data/obj/0592.png', 'data/obj/0056.png', 'data/obj/0434.png', 'data/obj/0323.png', 'data/obj/0578.png', 'data/obj/0173.png', 'data/obj/0379.png', 'data/obj/0308.png', 'data/obj/0181.png', 'data/obj/0435.png', 'data/obj/0100.png', 'data/obj/0677.png', 'data/obj/0519.png', 'data/obj/0441.png', 'data/obj/0010.png', 'data/obj/0443.png', 'data/obj/0700.png', 'data/obj/0051.png', 'data/obj/0553.png', 'data/obj/0669.png', 'data/obj/0523.png', 'data/obj/0511.png', 'data/obj/0320.png', 'data/obj/0458.png', 'data/obj/0247.png', 'data/obj/0002.png', 'data/obj/0071.png', 'data/obj/0537.png', 'data/obj/0600.png', 'data/obj/0203.png', 'data/obj/0744.png', 'data/obj/0485.png', 'data/obj/0551.png', 'data/obj/0515.png', 'data/obj/0266.png', 'data/obj/0271.png', 'data/obj/0715.png', 'data/obj/0149.png', 'data/obj/0023.png', 'data/obj/0136.png', 'data/obj/0666.png', 'data/obj/0128.png', 'data/obj/0626.png', 'data/obj/0169.png', 'data/obj/0300.png', 'data/obj/0679.png', 'data/obj/02

In [ ]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Izpis pretočnega predvajanja je skrajšan na toliko zadnjih vrstic: 5000.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.816745), count: 1, class_loss = 0.014226, iou_loss = 0.065658, total_loss = 0.079884 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000032, iou_loss = 0.000000, total_loss = 0.000032 
 total_bbox = 58179, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.000000), count: 1, class_loss = 0.000020, iou_loss = 0.000000, total_loss = 0.000020 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.699281), count: 1, class_loss = 0.794045, iou_loss = 0.222299, total_loss = 1.016344 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 58180, rewritten_bbox